In [0]:
import findspark

findspark.find()
findspark.init()


In [0]:
# importing sparksession
from pyspark.sql import SparkSession

# starting the spark session
spark = SparkSession.builder.master("local[*]").appName("Banking").getOrCreate()
spark


In [0]:
# load data
df = spark.read.csv("./BankTransactionDataset/Loan.csv", inferSchema=True, header=True)
df.show(10)


In [0]:
df.dtypes

In [0]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType,StringType

def get_digits(s):
    return ''.join([i for i in s if i.isdigit()])

get_digits_udf = udf(get_digits, StringType())

In [0]:
df = df.withColumn('Loan Amount', get_digits_udf(col('Loan Amount')).cast(IntegerType()))\
    .withColumnRenamed(' Debt Record','Debt Record')\
    .withColumn('Debt Record', get_digits_udf(col('Debt Record')).cast(IntegerType()))\
    .withColumnRenamed(' Returned Cheque','Returned Cheque')\
    .withColumnRenamed(' Dishonour of Bill','Dishonor of Bill')


In [0]:
df.show()

In [0]:
df.dtypes

In [0]:
# Number of loans in each category
from pyspark.sql import functions as f

df.groupBy("Loan Category").agg(f.count("Loan Category").alias("count")).orderBy(
    "count", ascending=False
).show()


In [0]:
# b. Number of people who have taken more than 1 lakh loan
# df.where('"Loan Amount" > "1,00,000"').show()
# df.filter(df['Loan Amount']> '1,00,000').agg(f.count('Loan Amount').alias('count of loan_amount_more_than_1lac')).show()
df.filter(df["Loan Amount"] > 100000).count()

In [0]:
# c. Number of people with income greater than 60000 rupees
df.filter(df['Income'] > 60000).count()

In [0]:
# d. Number of people with 2 or more returned cheques and income less than 50000
df.filter((df['Returned Cheque'] > 2) & (df['Income'] < 50000)).count()

In [0]:
# e. Number of people with 2 or more returned cheques and are single
df.filter((df['Returned Cheque'] > 2) & (df['Marital Status']=='SINGLE')).count()

In [0]:
# f. Number of people with expenditure over 50000 a month
df.filter(df['Expenditure'] > 50000).count()